In [2]:
import gym
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

In [13]:

def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

def epsilon_action(epsilon):
    random_value = np.random.random()
    if random_value<=epsilon:
        action = env.action_space.sample()
    else:
        Y=online_model.predict(state.reshape(1,-1))
        action = np.argmax(Y[0])
    return action


In [14]:


gamma=0.99
epsilon_decay = 0.005
learning_rate =0.002
max_e=1
min_e=0.0001
replay_batch_size=30
iterations=1000 

In [15]:
env = gym.make('LunarLander-v2')
state=env.reset()
online_model = initiate_NN(learning_rate)
memory=[]
episode_reward_rolling = []
episode_reward=[]

for episode in range(iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                y[non_terminal_samples] += np.multiply(gamma, \
                            online_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)

    episode_reward.append(current_episode_reward) 
    if len(memory)>=5000:
        memory = memory[-5000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[:20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  0  episode reward:  -202.09  rolling reward:  -202.09  Steps:  90  epsilon:  1.00
episode:  1  episode reward:  -392.17  rolling reward:  -297.13  Steps:  104  epsilon:  1.00
episode:  2  episode reward:  -209.56  rolling reward:  -267.94  Steps:  68  epsilon:  0.99
episode:  3  episode reward:  -264.51  rolling reward:  -267.08  Steps:  106  epsilon:  0.99
episode:  4  episode reward:  -426.48  rolling reward:  -298.96  Steps:  89  epsilon:  0.98
episode:  5  episode reward:  -133.48  rolling reward:  -271.38  Steps:  81  epsilon:  0.98
episode:  6  episode reward:  -128.63  rolling reward:  -250.99  Steps:  64  epsilon:  0.97
episode:  7  episode reward:  -115.01  rolling reward:  -233.99  Steps:  125  epsilon:  0.97
episode:  8  episode reward:  -118.94  rolling reward:  -221.21  Steps:  142  epsilon:  0.96
episode:  9  episode reward:  -183.58  rolling reward:  -217.44  Steps:  153  epsilon:  0.96
episode:  10  episode reward:  -96.10  rolling reward:  -206.41  Steps:  11

episode:  89  episode reward:  -136.91  rolling reward:  -120.59  Steps:  91  epsilon:  0.64
episode:  90  episode reward:  -29.97  rolling reward:  -119.60  Steps:  165  epsilon:  0.64
episode:  91  episode reward:  -284.06  rolling reward:  -121.38  Steps:  99  epsilon:  0.63
episode:  92  episode reward:  -79.70  rolling reward:  -120.94  Steps:  135  epsilon:  0.63
episode:  93  episode reward:  -67.03  rolling reward:  -120.36  Steps:  156  epsilon:  0.63
episode:  94  episode reward:  2.52  rolling reward:  -119.07  Steps:  176  epsilon:  0.63
episode:  95  episode reward:  -66.99  rolling reward:  -118.53  Steps:  141  epsilon:  0.62
episode:  96  episode reward:  -64.15  rolling reward:  -117.97  Steps:  119  epsilon:  0.62
episode:  97  episode reward:  -14.31  rolling reward:  -116.91  Steps:  96  epsilon:  0.62
episode:  98  episode reward:  -34.56  rolling reward:  -116.08  Steps:  134  epsilon:  0.61
episode:  99  episode reward:  -57.21  rolling reward:  -115.49  Steps:  

KeyboardInterrupt: 

In [10]:

for episode in range(196,iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    if episode>500:
        epsilon=1e-3
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                y[non_terminal_samples] += np.multiply(gamma, \
                            online_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)

    episode_reward.append(current_episode_reward) 
    if len(memory)>=5000:
        memory = memory[-5000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[:20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  196  episode reward:  62.80  rolling reward:  -45.67  Steps:  1000  epsilon:  0.38
episode:  197  episode reward:  -22.17  rolling reward:  -45.74  Steps:  128  epsilon:  0.37
episode:  198  episode reward:  6.87  rolling reward:  -44.88  Steps:  222  epsilon:  0.37
episode:  199  episode reward:  -11.24  rolling reward:  -44.39  Steps:  101  epsilon:  0.37
episode:  200  episode reward:  -82.65  rolling reward:  -43.97  Steps:  117  epsilon:  0.37
episode:  201  episode reward:  -33.60  rolling reward:  -43.65  Steps:  123  epsilon:  0.37
episode:  202  episode reward:  -17.80  rolling reward:  -43.00  Steps:  140  epsilon:  0.36
episode:  203  episode reward:  -70.49  rolling reward:  -42.42  Steps:  241  epsilon:  0.36
episode:  204  episode reward:  -65.99  rolling reward:  -39.57  Steps:  1000  epsilon:  0.36
episode:  205  episode reward:  -23.54  rolling reward:  -38.12  Steps:  128  epsilon:  0.36
episode:  206  episode reward:  77.58  rolling reward:  -37.17  Steps: 

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x0000029F050E3310>
Traceback (most recent call last):
  File "C:\Apps\Anaconda\lib\weakref.py", line 346, in remove
    self = selfref()
KeyboardInterrupt: 


KeyboardInterrupt: 

In [11]:

for episode in range(219,iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    if episode>500:
        epsilon=1e-3
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                y[non_terminal_samples] += np.multiply(gamma, \
                            online_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)

    episode_reward.append(current_episode_reward) 
    if len(memory)>=5000:
        memory = memory[-5000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[-20:])>200)>18 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  219  episode reward:  75.39  rolling reward:  -33.66  Steps:  1000  epsilon:  0.33
episode:  220  episode reward:  -218.84  rolling reward:  -35.35  Steps:  106  epsilon:  0.33
episode:  221  episode reward:  29.63  rolling reward:  -34.81  Steps:  1000  epsilon:  0.33
episode:  222  episode reward:  13.49  rolling reward:  -34.35  Steps:  1000  epsilon:  0.33
episode:  223  episode reward:  -205.03  rolling reward:  -35.55  Steps:  99  epsilon:  0.33
episode:  224  episode reward:  -40.90  rolling reward:  -35.17  Steps:  93  epsilon:  0.33
episode:  225  episode reward:  -214.40  rolling reward:  -36.69  Steps:  220  epsilon:  0.32
episode:  226  episode reward:  -42.59  rolling reward:  -37.51  Steps:  209  epsilon:  0.32
episode:  227  episode reward:  2.70  rolling reward:  -36.57  Steps:  1000  epsilon:  0.32
episode:  228  episode reward:  26.91  rolling reward:  -36.16  Steps:  241  epsilon:  0.32
episode:  229  episode reward:  86.07  rolling reward:  -34.38  Steps: 

episode:  308  episode reward:  -6.67  rolling reward:  16.32  Steps:  1000  epsilon:  0.21
episode:  309  episode reward:  185.21  rolling reward:  19.22  Steps:  992  epsilon:  0.21
episode:  310  episode reward:  145.64  rolling reward:  20.05  Steps:  826  epsilon:  0.21
episode:  311  episode reward:  -238.72  rolling reward:  19.47  Steps:  427  epsilon:  0.21
episode:  312  episode reward:  100.49  rolling reward:  20.33  Steps:  1000  epsilon:  0.21
episode:  313  episode reward:  1.08  rolling reward:  20.63  Steps:  107  epsilon:  0.21
episode:  314  episode reward:  210.92  rolling reward:  22.26  Steps:  744  epsilon:  0.21
episode:  315  episode reward:  190.77  rolling reward:  24.43  Steps:  1000  epsilon:  0.21
episode:  316  episode reward:  132.35  rolling reward:  25.76  Steps:  1000  epsilon:  0.21
episode:  317  episode reward:  186.67  rolling reward:  27.37  Steps:  573  epsilon:  0.21
episode:  318  episode reward:  202.69  rolling reward:  29.01  Steps:  562  e

episode:  397  episode reward:  202.49  rolling reward:  96.03  Steps:  610  epsilon:  0.14
episode:  398  episode reward:  -45.29  rolling reward:  93.38  Steps:  339  epsilon:  0.14
episode:  399  episode reward:  223.53  rolling reward:  94.60  Steps:  372  epsilon:  0.14
episode:  400  episode reward:  -28.47  rolling reward:  93.32  Steps:  500  epsilon:  0.14
episode:  401  episode reward:  39.08  rolling reward:  93.80  Steps:  125  epsilon:  0.13
episode:  402  episode reward:  -105.22  rolling reward:  92.65  Steps:  210  epsilon:  0.13
episode:  403  episode reward:  -136.37  rolling reward:  90.63  Steps:  128  epsilon:  0.13
episode:  404  episode reward:  -67.82  rolling reward:  92.05  Steps:  604  epsilon:  0.13
episode:  405  episode reward:  -14.54  rolling reward:  90.03  Steps:  169  epsilon:  0.13
episode:  406  episode reward:  -210.75  rolling reward:  88.32  Steps:  190  epsilon:  0.13
episode:  407  episode reward:  -11.00  rolling reward:  90.92  Steps:  253  e

episode:  486  episode reward:  253.73  rolling reward:  2.65  Steps:  407  epsilon:  0.09
episode:  487  episode reward:  199.00  rolling reward:  3.04  Steps:  518  epsilon:  0.09
episode:  488  episode reward:  265.42  rolling reward:  3.15  Steps:  435  epsilon:  0.09
episode:  489  episode reward:  222.58  rolling reward:  5.80  Steps:  374  epsilon:  0.09
episode:  490  episode reward:  214.33  rolling reward:  7.24  Steps:  772  epsilon:  0.09
episode:  491  episode reward:  233.73  rolling reward:  7.31  Steps:  658  epsilon:  0.09
episode:  492  episode reward:  253.34  rolling reward:  12.62  Steps:  447  epsilon:  0.09
episode:  493  episode reward:  197.18  rolling reward:  15.06  Steps:  896  epsilon:  0.09
episode:  494  episode reward:  258.52  rolling reward:  17.17  Steps:  546  epsilon:  0.08
episode:  495  episode reward:  258.43  rolling reward:  21.71  Steps:  497  epsilon:  0.08
episode:  496  episode reward:  -134.64  rolling reward:  18.82  Steps:  106  epsilon:

episode:  576  episode reward:  -164.96  rolling reward:  42.42  Steps:  56  epsilon:  0.00
episode:  577  episode reward:  -334.32  rolling reward:  39.41  Steps:  75  epsilon:  0.00
episode:  578  episode reward:  -249.61  rolling reward:  37.11  Steps:  72  epsilon:  0.00
episode:  579  episode reward:  -56.85  rolling reward:  36.96  Steps:  152  epsilon:  0.00
episode:  580  episode reward:  -213.46  rolling reward:  32.64  Steps:  90  epsilon:  0.00
episode:  581  episode reward:  -265.37  rolling reward:  27.49  Steps:  81  epsilon:  0.00
episode:  582  episode reward:  -188.68  rolling reward:  24.15  Steps:  125  epsilon:  0.00
episode:  583  episode reward:  -299.89  rolling reward:  19.10  Steps:  371  epsilon:  0.00
episode:  584  episode reward:  -445.79  rolling reward:  12.46  Steps:  380  epsilon:  0.00
episode:  585  episode reward:  -346.82  rolling reward:  6.39  Steps:  242  epsilon:  0.00
episode:  586  episode reward:  -343.00  rolling reward:  0.42  Steps:  143  

episode:  663  episode reward:  -129.99  rolling reward:  -208.73  Steps:  1000  epsilon:  0.00
episode:  664  episode reward:  -206.18  rolling reward:  -210.14  Steps:  1000  epsilon:  0.00
episode:  665  episode reward:  -145.36  rolling reward:  -211.54  Steps:  1000  epsilon:  0.00
episode:  666  episode reward:  -110.45  rolling reward:  -211.86  Steps:  119  epsilon:  0.00
episode:  667  episode reward:  -147.31  rolling reward:  -212.94  Steps:  1000  epsilon:  0.00
episode:  668  episode reward:  -113.10  rolling reward:  -212.78  Steps:  115  epsilon:  0.00
episode:  669  episode reward:  -148.51  rolling reward:  -211.85  Steps:  1000  epsilon:  0.00
episode:  670  episode reward:  -157.53  rolling reward:  -209.73  Steps:  1000  epsilon:  0.00
episode:  671  episode reward:  -180.36  rolling reward:  -209.23  Steps:  804  epsilon:  0.00
episode:  672  episode reward:  -153.01  rolling reward:  -209.20  Steps:  1000  epsilon:  0.00
episode:  673  episode reward:  -169.15  ro

KeyboardInterrupt: 

In [12]:
online_model.save_weights('weights_singleDQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5')

import pickle

with open('weights_singleDQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)


In [167]:
rward = pickle.load( open( 'weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', "rb" ) )

In [168]:
rward

{'rewards': [-131.3223570990876, -248.23258991485326, -133.48636655895385],
 'rolling_reward': [-131.3223570990876,
  -189.77747350697044,
  -171.01377119096492]}

In [163]:
episode_reward

[]

In [152]:
Y_target.shape

(5, 4)

In [142]:
batch[terminal_samples].shape

(0, 5)

In [125]:



# i:index for sample [state,action,reward,new_state,done]
for i,sample in enumerate(batch):
    terminal_sample = sample[4]
    new_state_sample = sample[3]
    state_sample = sample[0]
    X[i]=state_sample
    action_sample = sample[1]
    Y_target[i] = online_model.predict(state_sample.reshape(1,-1))[0]
    if terminal:
        #terminal then targetY is reward
        Y_target[i][action_sample] = sample[2]
    else:
        #use online weights to select argmax action under new_state

        online_qvalues_batch =online_model.predict(new_state_sample.reshape(1,-1)) 
        max_action = np.argmax(online_qvalues_batch)
        #use target weights to calculate Qvalue
        #non-terminal then targetY is reward +gamma*new Qvalue
        Y_target[i][action_sample] = sample[2] + gamma * target_model.predict(new_state_sample.reshape(1,-1)).reshape(4,)[max_action]

online_model.fit(np.array(X), np.array(Y_target), verbose=0)


100

In [ ]:
Notice that the selection of the action, in the argmax, is


In [91]:
online_qvalues_batch

[array([[ 0.18740644, -0.2721934 ,  0.2233217 , -0.45678362]],
       dtype=float32),
 array([[ 0.19613695, -0.27545747,  0.20470577, -0.46681872]],
       dtype=float32),
 array([[ 0.18746991, -0.25133634,  0.19952038, -0.4359238 ]],
       dtype=float32),
 array([[ 0.19334945, -0.2700162 ,  0.19816679, -0.45707738]],
       dtype=float32),
 array([[ 0.19308917, -0.27386588,  0.21208042, -0.46272066]],
       dtype=float32)]

In [113]:
np.array(X)

array([[-0.00213823,  1.4003743 , -0.21659775, -0.4687014 ,  0.0024845 ,
         0.04906261,  0.        ,  0.        ],
       [-0.00846777,  1.3651963 , -0.20548455, -0.5481141 ,  0.00758275,
         0.00502167,  0.        ,  0.        ],
       [-0.01887293,  1.3004851 , -0.22156799, -0.61512053,  0.00625335,
         0.04479944,  0.        ,  0.        ],
       [-0.01250668,  1.3402183 , -0.20310256, -0.5355056 ,  0.00488083,
        -0.02924636,  0.        ,  0.        ],
       [-0.00641537,  1.3775289 , -0.21631236, -0.5210343 ,  0.00733169,
         0.04846335,  0.        ,  0.        ]], dtype=float32)

In [112]:
Y_target

[array([-1.2552428 , -0.2741652 ,  0.2263453 , -0.45686692], dtype=float32),
 array([ 0.19308917, -0.27386588,  0.21208042, -0.6722371 ], dtype=float32),
 array([ 0.1933487, -0.2546284,  5.1014786, -0.445849 ], dtype=float32),
 array([-0.91460276, -0.27277392,  0.20281965, -0.4571485 ], dtype=float32),
 array([ 0.18904   , -0.27001685,  0.21992686, -0.7534547 ], dtype=float32)]

In [ ]:

class DDQL:
    def __init__(self, nS, nA):
        self.nS = nS
        self.nA = nA
        self.epsilon = 1
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9993
        self.gamma = 0.99
        self.learning_rate = 0.0001
        self.epochs = 1
        self.verbose = 0
        self.minibatch_size = 30
        self.memory = deque(maxlen=5000)
        self.model = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model = Sequential()

        # Add 2 hidden layers with 64 nodes each
        model.add(Dense(64, input_dim=self.nS, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.nA, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, s, a, r, s_prime, done):
        self.memory.append((s, a, r, s_prime, done))

    def target_model_update(self):
        self.target_model.set_weights(self.model.get_weights())

    def selectAction(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.nA)
        q = self.model.predict(s)
        return np.argmax(q[0])

    def replay(self):
        # Vectorized method for experience replay
        minibatch = random.sample(self.memory, self.minibatch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])

        # If minibatch contains any non-terminal states, use separate update rule for those states
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            # Non-terminal update rule
            y[not_done_indices] += np.multiply(self.gamma, \
                        predict_sprime_target[not_done_indices, \
                        np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(self.minibatch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=self.epochs, verbose=self.verbose)

    def replayIterative(self):
        # Iterative method - this performs the same function as replay() but is not vectorized 
        s_list = []
        y_state_list = []
        minibatch = random.sample(self.memory, self.minibatch_size)
        for s, a, r, s_prime, done in minibatch:
            s_list.append(s)
            y_action = r
            if not done:
                y_action = r + self.gamma * np.amax(self.model.predict(s_prime)[0])

            print(y_action)
            
            y_state = self.model.predict(s)
            y_state[0][a] = y_action
            y_state_list.append(y_state)
        self.model.fit(np.squeeze(s_list), np.squeeze(y_state_list), batch_size=self.minibatch_size, epochs=1, verbose=0)

if __name__ == "__main__":
    np.set_printoptions(precision=2)
    env = gym.make('LunarLander-v2')
    env = wrappers.Monitor(env, tdir, force=True, video_callable=False)

    nS = env.observation_space.shape[0]
    nA = env.action_space.n

    agent = DDQL(nS, nA)

    # Set to true to use saved model
    episodes = 10000

    # Cumulative reward
    reward_avg = deque(maxlen=100)

    for e in range(episodes):
        episode_reward = 0
        s = env.reset()
        s = np.reshape(s, [1, nS])

        for time in range(1000):

            # Query next action from learner and perform action
            a = agent.selectAction(s)
            s_prime, r, done, info = env.step(a)

            # Add cumulative reward
            episode_reward += r

            # Reshape new state
            s_prime = np.reshape(s_prime, [1, nS])

            # Add experience to memory
            agent.add_memory(s, a, r, s_prime, done)

            # Set current state to new state
            s = s_prime

            #Perform experience replay if memory length is greater than minibatch length
   
            if len(agent.memory) > agent.minibatch_size:
                agent.replay()

            # If episode is done, exit loop
            if done:
                agent.target_model_update()
                break


        # epsilon decay
        if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay

        # Running average of past 100 episodes
        reward_avg.append(episode_reward)
        print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)
        
        # with open('trained_agent.txt', 'a') as f:
        #     f.write(str(np.average(reward_avg)) + '\n')

    env.close()
#     gym.upload(tdir, api_key='sk_EJo79Jo0RsSEA4EF6Cp5mg')


#agent.model.save_weights('weights.h5')
#agent.model.load_weights('weights.h5')

episode:  0  score:  -101.20  avg_score:  -101.20  frames:  104  epsilon:  1.00
episode:  1  score:  -147.99  avg_score:  -124.59  frames:  74  epsilon:  1.00
episode:  2  score:  -95.67  avg_score:  -114.95  frames:  74  epsilon:  1.00
episode:  3  score:  -218.30  avg_score:  -140.79  frames:  94  epsilon:  1.00
episode:  4  score:  -231.98  avg_score:  -159.03  frames:  86  epsilon:  1.00
episode:  5  score:  -161.32  avg_score:  -159.41  frames:  68  epsilon:  1.00
episode:  6  score:  -96.64  avg_score:  -150.44  frames:  66  epsilon:  1.00
episode:  7  score:  -141.43  avg_score:  -149.32  frames:  97  epsilon:  0.99
episode:  8  score:  -480.81  avg_score:  -186.15  frames:  102  epsilon:  0.99
episode:  9  score:  -371.18  avg_score:  -204.65  frames:  92  epsilon:  0.99
episode:  10  score:  -161.56  avg_score:  -200.73  frames:  95  epsilon:  0.99
episode:  11  score:  -80.20  avg_score:  -190.69  frames:  61  epsilon:  0.99
episode:  12  score:  -332.47  avg_score:  -201.60 

episode:  103  score:  -168.96  avg_score:  -179.65  frames:  118  epsilon:  0.93
episode:  104  score:  -64.43  avg_score:  -177.97  frames:  91  epsilon:  0.93
episode:  105  score:  -200.70  avg_score:  -178.36  frames:  72  epsilon:  0.93
episode:  106  score:  32.45  avg_score:  -177.07  frames:  121  epsilon:  0.93
episode:  107  score:  -260.25  avg_score:  -178.26  frames:  117  epsilon:  0.93
episode:  108  score:  -110.97  avg_score:  -174.56  frames:  80  epsilon:  0.93
episode:  109  score:  -253.15  avg_score:  -173.38  frames:  94  epsilon:  0.93
episode:  110  score:  -125.12  avg_score:  -173.02  frames:  88  epsilon:  0.93
episode:  111  score:  -189.74  avg_score:  -174.11  frames:  72  epsilon:  0.92
episode:  112  score:  -140.07  avg_score:  -172.19  frames:  94  epsilon:  0.92
episode:  113  score:  -304.20  avg_score:  -173.17  frames:  71  epsilon:  0.92
episode:  114  score:  -128.94  avg_score:  -173.02  frames:  72  epsilon:  0.92
episode:  115  score:  -294.

episode:  204  score:  -298.51  avg_score:  -150.87  frames:  97  epsilon:  0.87
episode:  205  score:  -77.35  avg_score:  -149.64  frames:  146  epsilon:  0.87
episode:  206  score:  -124.22  avg_score:  -151.21  frames:  68  epsilon:  0.87
episode:  207  score:  -40.81  avg_score:  -149.01  frames:  144  epsilon:  0.86
episode:  208  score:  -83.08  avg_score:  -148.73  frames:  67  epsilon:  0.86
episode:  209  score:  -268.33  avg_score:  -148.89  frames:  92  epsilon:  0.86
episode:  210  score:  -124.07  avg_score:  -148.88  frames:  105  epsilon:  0.86
episode:  211  score:  -123.48  avg_score:  -148.21  frames:  122  epsilon:  0.86
episode:  212  score:  -143.46  avg_score:  -148.25  frames:  80  epsilon:  0.86
episode:  213  score:  -86.07  avg_score:  -146.07  frames:  96  epsilon:  0.86
episode:  214  score:  -86.35  avg_score:  -145.64  frames:  94  epsilon:  0.86
episode:  215  score:  -55.86  avg_score:  -143.25  frames:  93  epsilon:  0.86
episode:  216  score:  -58.61 